In [94]:
import pandas as pd 
import ast
from ast import literal_eval
import numpy as np
from pandas import DataFrame

#setting the file we want to read
file = "train.csv"

#reading from CSV
df = pd.read_csv(file, header= 0, quotechar='"', sep=',', encoding = "ISO-8859-1",keep_default_na=True, 
                dtype = { 'belongs_to_collection' : 'category',  'genres' : 'category', 'original_language' : 'category', 'production_companies' : 'category', 'production_countries' : 'category',  'spoken_languages' : 'category', 'Keywords' : 'category', 'cast' : 'category', 'crew' : 'category'},
                na_values = [0]
                )

#setting the list to be edited
dataList = { 'belongs_to_collection',  'genres', 'production_companies', 'production_countries',  'spoken_languages', 'Keywords', 'cast', 'crew'}
dataID = { 'belongs_to_collection' : 'id',  'genres': 'id', 'production_companies': 'id', 'production_countries': 'iso_3166_1',  'spoken_languages': 'iso_639_1', 'Keywords': 'id', 'cast': 'id', 'crew': 'id'}
dataNeeded = ['id', 'belongs_to_collection', 'budget', 'genres', 'original_language',
       'popularity', 'production_companies', 'production_countries',
       'release_date', 'runtime', 'spoken_languages', 'Keywords',
       'cast', 'crew', 'revenue']

#dropping the columns that was not needed
for col in df.keys():
    if not (col in dataNeeded) :
        df = df.drop(columns = col)

#filling empty list with []
for col in dataList :
    try:
        df[col] = df[col].cat.add_categories('[]',inplace = False)
    except:
        pass
    df[col].fillna( '[]' , inplace = True) 

#changing a sting of list into list
for col in dataList :
    df[[col]] = df[[col]].applymap(literal_eval)     

#changing the dict in the list into ID only
for col in dataList : 
    for row in range(len(df)) :
        for num in range(len(df[col][row])) :
            if not (isinstance(df[col][row][num], int) or isinstance(df[col][row][num], str)):
                df[col][row][num] = df[col][row][num][dataID[col]]

uniqueSpokenLanguage = df['spoken_languages'].apply(pd.Series).stack().value_counts().keys().values.tolist()
for row in range(len(df)) :
    for num in range(len(df['spoken_languages'][row])):
        df['spoken_languages'][row][num] = uniqueSpokenLanguage.index(df.spoken_languages[row][num])
        
df.original_language = df.original_language.replace(uniqueSpokenLanguage,range(len(uniqueSpokenLanguage)))

uniqueCountries = df['production_countries'].apply(pd.Series).stack().value_counts().keys().values.tolist()

for row in range(len(df)) :
    for num in range(len(df['production_countries'][row])):
        df['production_countries'][row][num] = uniqueCountries.index(df.production_countries[row][num])


df.head()

,id,belongs_to_collection,budget,genres,original_language,popularity,production_companies,production_countries,release_date,runtime,spoken_languages,Keywords,cast,crew,revenue
0,1,[313576],14000000.0,[35],0,6.575393,"[4, 60, 8411]",[0],2/20/15,93.0,[0],"[4379, 9663, 11830, 179431]","[52997, 64342, 54729, 36801, 54812, 94098, 115...","[1449071, 3227, 347335, 347335, 57822, 1451395...",12314651
1,2,[107674],40000000.0,"[35, 18, 10751, 10749]",0,8.248895,[2],[0],8/6/2004,113.0,[0],"[2505, 4263, 6038, 13072]","[1813, 5823, 1210, 655, 33656, 62064, 68287, 1...","[1201, 1214, 4500, 8851, 12970, 38415, 38416, ...",95149435
2,3,[],3300000.0,[18],0,64.299990,"[2266, 3172, 32157]",[0],10/10/2014,105.0,[0],"[1416, 1523, 1640, 2176, 14512, 14819, 33896, ...","[996701, 18999, 129104, 970216, 223012, 159366...","[494, 6745, 9250, 23783, 23786, 53181, 52443, ...",13092000
3,4,[],1200000.0,"[53, 18]",8,3.174936,[],[5],3/9/2012,122.0,"[0, 8]","[10092, 10540, 11734, 14536, 14636, 208364, 22...","[35068, 85047, 1021524, 1093644, 86033, 92686,...","[95510, 95510, 95510]",16000000
4,5,[],NaN,"[28, 53]",11,1.148070,[],[15],2/5/2009,118.0,[11],[],"[84751, 64453, 84752, 1130534]","[84749, 84749]",3923970


In [95]:
# changing the time into numerical value

from datetime import datetime
from dateutil.parser import parse
import time
import datetime

x = datetime.datetime(1970, 1, 1)

for count in range(len(df)) :
    date = df.release_date[count]
    if (isinstance(date, str))and(date[-3] == "/"):
        if(int(date[-2], 10) < 2):
            df.release_date[count] = date[:-2] + '20' + date[-2:]
        else:
            df.release_date[count] = date[:-2] + '19' + date[-2:]
        print(date)
        
    if (isinstance(date, str)):
        print(date)
        df.release_date[count] = datetime.datetime.strptime(df.release_date[count], '%m/%d/%Y')

    if  isinstance(df.release_date[count], datetime.datetime):
        print(count, df.release_date[count])
        if(df.release_date[count].year >= 1970):
            df.release_date[count] =(df.release_date[count]-x).days + 365.25*15
        else:
            df.release_date[count] = (df.release_date[count].year-1955)*365.25 + (df.release_date[count].month-1)*(365.25/12)+(df.release_date[count].day-1)
        print(df.release_date[count])

C:\Users\isabe\Anaconda3\envs\env1\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\isabe\Anaconda3\envs\env1\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\isabe\Anaconda3\envs\env1\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


2/20/15
2/20/15
0 2015-02-20 00:00:00
21964.75
8/6/2004
1 2004-08-06 00:00:00
18114.75
10/10/2014
2 2014-10-10 00:00:00
21831.75
3/9/2012
3 2012-03-09 00:00:00
20886.75
2/5/2009
4 2009-02-05 00:00:00
19758.75
8/6/1987
5 1987-08-06 00:00:00
11904.75
8/30/12
8/30/12
6 2012-08-30 00:00:00
21060.75
1/15/04
1/15/04
7 2004-01-15 00:00:00
17910.75
2/16/96
2/16/96
8 1996-02-16 00:00:00
15020.75


C:\Users\isabe\Anaconda3\envs\env1\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


4/16/03
4/16/03
9 2003-04-16 00:00:00
17636.75
11/21/76
11/21/76
10 1976-11-21 00:00:00
7994.75
7/10/1987
11 1987-07-10 00:00:00
11877.75
9/15/99
9/15/99
12 1999-09-15 00:00:00
16327.75
3/4/2005
13 2005-03-04 00:00:00
18324.75
6/20/02
6/20/02
14 2002-06-20 00:00:00
17336.75
10/6/2010
15 2010-10-06 00:00:00
20366.75
8/4/2005
16 2005-08-04 00:00:00
18477.75
12/25/13
12/25/13
17 2013-12-25 00:00:00
21542.75
2/2/2011
18 2011-02-02 00:00:00
20485.75
8/2/2005
19 2005-08-02 00:00:00
18475.75
4/3/1998
20 1998-04-03 00:00:00
15797.75
8/13/82
8/13/82
21 1982-08-13 00:00:00
10085.75
7/28/12
7/28/12
22 2012-07-28 00:00:00
21027.75
9/2/2011
23 2011-09-02 00:00:00
20697.75
9/9/2006
24 2006-09-09 00:00:00
18878.75
10/23/92
10/23/92
25 1992-10-23 00:00:00
13809.75
9/8/1997
26 1997-09-08 00:00:00
15590.75
9/7/2013
27 2013-09-07 00:00:00
21433.75
8/8/1986
28 1986-08-08 00:00:00
11541.75
4/7/2002
29 2002-04-07 00:00:00
17262.75
11/2/2011
30 2011-11-02 00:00:00
20758.75
5/2/2005
31 2005-05-02 00:00:00
183

C:\Users\isabe\Anaconda3\envs\env1\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


41 1967-06-22 00:00:00
4556.1875
3/3/2000
42 2000-03-03 00:00:00
16497.75
9/5/2014
43 2014-09-05 00:00:00
21796.75
2/4/2005
44 2005-02-04 00:00:00
18296.75
10/9/2009
45 2009-10-09 00:00:00
20004.75
3/23/06
3/23/06
46 2006-03-23 00:00:00
18708.75
12/25/15
12/25/15
47 2015-12-25 00:00:00
22272.75
3/24/17
3/24/17
48 2017-03-24 00:00:00
22727.75
11/17/94
11/17/94
49 1994-11-17 00:00:00
14564.75
1/30/08
1/30/08
50 2008-01-30 00:00:00
19386.75
7/1/1965
51 1965-07-01 00:00:00
3835.125
6/24/80
6/24/80
52 1980-06-24 00:00:00
9305.75
1/15/88
1/15/88
53 1988-01-15 00:00:00
12066.75
2/19/16
2/19/16
54 2016-02-19 00:00:00
22328.75
7/2/1993
55 1993-07-02 00:00:00
14061.75
8/5/2009
56 2009-08-05 00:00:00
19939.75
6/20/97
6/20/97
57 1997-06-20 00:00:00
15510.75
6/24/08
6/24/08
58 2008-06-24 00:00:00
19532.75
3/8/2002
59 2002-03-08 00:00:00
17232.75
11/14/06
11/14/06
60 2006-11-14 00:00:00
18944.75
5/30/56
5/30/56
61 1956-05-30 00:00:00
516.0
1/16/98
1/16/98
62 1998-01-16 00:00:00
15720.75
7/28/89
7/28

In [98]:
#check if all time is in float
for count in range(len(df)) :
    if  not isinstance(df.release_date[count], float):
        print (count,df.release_date[count])

In [96]:
# change the empty list back to nan
for x in dataList:
    df[x] = df[x].apply(lambda y: np.nan if len(y)==0 else y)
df.head()

,id,belongs_to_collection,budget,genres,original_language,popularity,production_companies,production_countries,release_date,runtime,spoken_languages,Keywords,cast,crew,revenue
0,1,[313576],14000000.0,[35],0,6.575393,"[4, 60, 8411]",[0],21964.8,93.0,[0],"[4379, 9663, 11830, 179431]","[52997, 64342, 54729, 36801, 54812, 94098, 115...","[1449071, 3227, 347335, 347335, 57822, 1451395...",12314651
1,2,[107674],40000000.0,"[35, 18, 10751, 10749]",0,8.248895,[2],[0],18114.8,113.0,[0],"[2505, 4263, 6038, 13072]","[1813, 5823, 1210, 655, 33656, 62064, 68287, 1...","[1201, 1214, 4500, 8851, 12970, 38415, 38416, ...",95149435
2,3,NaN,3300000.0,[18],0,64.299990,"[2266, 3172, 32157]",[0],21831.8,105.0,[0],"[1416, 1523, 1640, 2176, 14512, 14819, 33896, ...","[996701, 18999, 129104, 970216, 223012, 159366...","[494, 6745, 9250, 23783, 23786, 53181, 52443, ...",13092000
3,4,NaN,1200000.0,"[53, 18]",8,3.174936,NaN,[5],20886.8,122.0,"[0, 8]","[10092, 10540, 11734, 14536, 14636, 208364, 22...","[35068, 85047, 1021524, 1093644, 86033, 92686,...","[95510, 95510, 95510]",16000000
4,5,NaN,NaN,"[28, 53]",11,1.148070,NaN,[15],19758.8,118.0,[11],NaN,"[84751, 64453, 84752, 1130534]","[84749, 84749]",3923970


In [97]:
df.head()

,id,belongs_to_collection,budget,genres,original_language,popularity,production_companies,production_countries,release_date,runtime,spoken_languages,Keywords,cast,crew,revenue
0,1,[313576],14000000.0,[35],0,6.575393,"[4, 60, 8411]",[0],21964.8,93.0,[0],"[4379, 9663, 11830, 179431]","[52997, 64342, 54729, 36801, 54812, 94098, 115...","[1449071, 3227, 347335, 347335, 57822, 1451395...",12314651
1,2,[107674],40000000.0,"[35, 18, 10751, 10749]",0,8.248895,[2],[0],18114.8,113.0,[0],"[2505, 4263, 6038, 13072]","[1813, 5823, 1210, 655, 33656, 62064, 68287, 1...","[1201, 1214, 4500, 8851, 12970, 38415, 38416, ...",95149435
2,3,NaN,3300000.0,[18],0,64.299990,"[2266, 3172, 32157]",[0],21831.8,105.0,[0],"[1416, 1523, 1640, 2176, 14512, 14819, 33896, ...","[996701, 18999, 129104, 970216, 223012, 159366...","[494, 6745, 9250, 23783, 23786, 53181, 52443, ...",13092000
3,4,NaN,1200000.0,"[53, 18]",8,3.174936,NaN,[5],20886.8,122.0,"[0, 8]","[10092, 10540, 11734, 14536, 14636, 208364, 22...","[35068, 85047, 1021524, 1093644, 86033, 92686,...","[95510, 95510, 95510]",16000000
4,5,NaN,NaN,"[28, 53]",11,1.148070,NaN,[15],19758.8,118.0,[11],NaN,"[84751, 64453, 84752, 1130534]","[84749, 84749]",3923970
